In [21]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
df = pd.read_csv("billboard100_data.csv")

In [22]:
df['title'] = df['title'].map(str.lower) # converting the entries to lower
df['author'] = df['author'].map(str.lower) # converting the entries to lower
df

,title,author
0,flowers,miley cyrus
1,kill bill,sza
2,last night,morgan wallen
3,unholy,sam smith & kim petras
4,creepin',"metro boomin, the weeknd & 21 savage"
...,...,...
95,yandel 150,yandel & feid
96,heart to heart,mac demarco
97,next thing you know,jordan davis
98,never gonna not dance again,p!nk


In [23]:
#from fuzzywuzzy import process
#def check_mistakes(song_name, reference_database, title_column, artist_column):
#    best_match, score = process.extractOne(song_name, reference_database[title_column].values)
#    if score >= 70:
#        artist = reference_database[reference_database[title_column] == best_match][artist_column][0]
#        answer = input("f\Did you mean {best_match} by {artist}? Y/N: ")
#        #answer = input(“”).lower().title()
#        if answer == "Y":
#            return best_match
#        if answer == "N":
#            pass
#    else:
#        print("error")

In [24]:
x = input('Enter a song you like: ')
x_artist = df.loc[df['title'].map(str.lower)  == x.lower(), 'author'] # artist corresponding to song x
artist_question = input('Did you mean ' + x.lower().title() + ' by ' + x_artist.to_string(index=False).title() + '? (y/n)')
#x_artist.to_string(index=False).title() converts the series x_artist to a string and then uppers the first alphabets
#x.lower().title()  lowers the entries and then uppers the first alphabets
if artist_question == 'y':
    if x.lower() in df.values:   # converts te entered x into all lower case
        rec_song = df['title'].sample().to_string(index=False) # recommendation song. I just removed the indexes as they are not required as the output  
        rec_artist = df.loc[df['title'] == rec_song, 'author'].to_string(index=False) # artist of the recommended song
        # to_string(index=False) lets you output without index
        if x != rec_song:
            print("You might like this song: " + rec_song.title() + " by " + rec_artist.title())  # .title() makes the first alphabet in every word to upper   
    else:
        print("\nThis song does not exists in the data set.")
else:
    print("\nThis song does not exists in the data set.")
        

Enter a song you like: Flowers
Did you mean Flowers by Miley Cyrus? (y/n)y
You might like this song: Que Vuelvas by Carin Leon X Grupo Frontera


# Spotify dataframe

In [25]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))


Playlists

In [26]:
# Rock classics playlist link: https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=013db49871054a73   
# Rock classics playlist ID: 37i9dQZF1DWXRqgorJj26U 
rock_playlist_id = "37i9dQZF1DWXRqgorJj26U"
playlist_rock = sp.user_playlist_tracks("spotify", rock_playlist_id)
#import pprint
#pprint.pprint(playlist_rock) # in case you want to check the playlist

jazz_playlist_id = "37i9dQZF1DXbITWG1ZJKYt"
pop_playlist_id = "7iUGicmg9AkkFhkLiYqYpA"
blues_playlist_id = "37i9dQZF1DXd9rSDyQguIk"
techno_playlist_id = "18vUeZ9BdtMRNV6gI8RnR6"
allgenre_playlist_id = "52oPdG5WK1KcMf78FvFXIa"

electronic_playlist_id = "37i9dQZF1DX0wMD4IoQ5aJ"
house_playlist_id = "2otQLmbi8QWHjDfq3eL0DC"    
trend23_playlist_id = "5JNNpb0q7ZIM3s3j355Anr"    
afro_playlist_id = "37i9dQZF1DWYs2pvwxWA7l"
rap_playlist_id = "4riovLwMCrY3q0Cd4e0Sqp"    
brazil_playlist_id = "37i9dQZF1DX9FSYmdU9mbT"
spanish_playlist_id = "0dexh2j27EfJtsncx1vjhW"
kpop_playlist_id = "14dxceLGBZkRAj6WJTfkpj"    


playlist_ids = [rock_playlist_id, jazz_playlist_id, pop_playlist_id, blues_playlist_id, 
                techno_playlist_id, allgenre_playlist_id, electronic_playlist_id, house_playlist_id,
               trend23_playlist_id, afro_playlist_id, rap_playlist_id, brazil_playlist_id, 
               spanish_playlist_id, kpop_playlist_id]

In [27]:
# Extracting many songs from the playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [28]:
# Retrieve all tracks from the playlists
import time
tracks = []
counter = 0
for playlist_id in playlist_ids:
    tracks += get_playlist_tracks('spotify', playlist_id)
    counter += 1
    if counter % 50 == 0:
        time.sleep(5) # sleep for 5 seconds every 50 playlists

In [29]:
# extract song ids from the playlist
list_of_audio_features=[]
counter1 = 0
for item in range(0,len(tracks)):
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])
    counter1 += 1
    if counter % 50 == 0:
        time.sleep(5) # sleep for 5 seconds every 50 playlists

In [30]:
# creating the data frame
df_spotify=pd.DataFrame(list_of_audio_features)    
df_spotify=df_spotify[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df_spotify

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.392,0.476,-7.583,0.0249,0.39700,0.000163,0.3090,0.234,78.031,1xsYj84j7hUDDnTTerGWlH,267597
1,0.548,0.691,-13.553,0.0369,0.01500,0.106000,0.1440,0.667,137.145,4KfSdst7rW39C0sfhArdrz,261933
2,0.413,0.881,-5.541,0.0367,0.00006,0.000308,0.0805,0.364,158.066,5UWwZ5lm5PKu6eKsHAGxOk,250547
3,0.532,0.887,-3.757,0.0335,0.07780,0.000214,0.2940,0.795,122.511,37ZJ0p5Jm13JPevGcx4SkF,249293
4,0.545,0.640,-9.937,0.2240,0.00249,0.030400,0.1960,0.606,80.544,70YvYr2hGlS01bKRIho1HM,230480
...,...,...,...,...,...,...,...,...,...,...,...
3510,0.882,0.797,-3.299,0.0819,0.08210,0.000000,0.1130,0.824,105.025,4fsQ0K37TOXa3hEQfjEic1,184444
3511,0.625,0.712,-5.996,0.0539,0.02290,0.000000,0.2520,0.398,163.015,3IelG5zYpWWCZIH4cqWlPV,203557
3512,0.631,0.787,-5.718,0.0640,0.10400,0.000000,0.4600,0.452,99.991,5e7WT8HDdFlHQBO3ZbKvVj,184560
3513,0.804,0.771,-4.067,0.0433,0.35700,0.000003,0.1080,0.739,126.956,65FftemJ1DbbZ45DUfHJXE,212253


In [31]:
df_spotify.to_csv('spotify_data.csv', index=False)